<a href="https://colab.research.google.com/github/leadeeerq/data-science/blob/main/230227_DataCleaningIdentifyingHiddenMissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [335]:
import pandas as pd

##Loading the data

In [336]:
df = pd.read_csv('/content/drive/MyDrive/230222_Master 12 Advanced Projects/Project_03_Materials/movies_metadata.csv', low_memory = False)

In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

###Stringified data

In [338]:
df.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [339]:
df.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

###Dropping irrelevant columns

In [340]:
df.adult.value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [341]:
df.drop(columns = ['adult'], inplace = True)

In [342]:
df.drop(columns = ['imdb_id', 'original_title', 'video', 'homepage'], inplace = True)

##Handling stringified JSON columns, ast library

In [343]:
import json
import ast

In [344]:
json_cols = ['belongs_to_collection', 'genres', 'production_countries', 'production_companies', 'spoken_languages']

In [345]:
df.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

###Replacing single quotes with double ' -> "

In [346]:
example1 = "{'dog': 3, 'cat': 5}"

In [347]:
# json.loads(example1)

In [348]:
example2 = '{"dog": 3, "cat": 5}'
json.loads(example2)

{'dog': 3, 'cat': 5}

In [349]:
json.loads(example1.replace("'", '"'))

{'dog': 3, 'cat': 5}

###That leads to having a list

In [350]:
df.genres.apply(lambda x: json.loads(x.replace("'", '"')))[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

###Using ast.literal_eval - it evaluates a string containing a python expression

In [351]:
ast.literal_eval(example1)

{'dog': 3, 'cat': 5}

In [352]:
ast.literal_eval(example2)

{'dog': 3, 'cat': 5}

In [353]:
df.genres.apply(ast.literal_eval)[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [354]:
# df.genres = df.genres.apply(ast.literal_eval)

###Applying the ast.literal_eval function to all columns

It doesn't work for values different than strings

In [355]:
# df.loc[:, json_cols].apply(ast.literal_eval, axis = 0)

In [356]:
df.belongs_to_collection.apply(lambda x: isinstance(x, str))

0         True
1        False
2         True
3        False
4         True
         ...  
45461    False
45462    False
45463    False
45464    False
45465    False
Name: belongs_to_collection, Length: 45466, dtype: bool

In [357]:
import numpy as np

In [358]:
# df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [359]:
df.belongs_to_collection

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [360]:
for col in json_cols:
  df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [361]:
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,4.0,1.0
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0
45464,NaN,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0


##Flattening the nested columns

In [362]:
df.belongs_to_collection[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

###Belongs_to_collection - take only the value of the key 'name'

In [363]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

In [364]:
df.belongs_to_collection.value_counts(dropna = False).head(10)

NaN                                       40975
The Bowery Boys                              29
Totò Collection                              27
James Bond Collection                        26
Zatôichi: The Blind Swordsman                26
The Carry On Collection                      25
Pokémon Collection                           22
Charlie Chan (Sidney Toler) Collection       21
Godzilla (Showa) Collection                  16
Uuno Turhapuro                               15
Name: belongs_to_collection, dtype: int64

###Genres - join values of keys 'name' with '|'

In [365]:
df.genres[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [366]:
df.genres = df.genres.apply(lambda x: '|'.join(i['name'] for i in x))

In [367]:
df.genres[0]

'Animation|Comedy|Family'

In [368]:
df.genres.value_counts(dropna = False).head(10)

Drama                   5000
Comedy                  3621
Documentary             2723
                        2442
Drama|Romance           1301
Comedy|Drama            1135
Horror                   974
Comedy|Romance           930
Comedy|Drama|Romance     593
Drama|Comedy             532
Name: genres, dtype: int64

###Replacing empty strings with np.nan

In [369]:
df.genres.replace('', np.nan, inplace = True)

###Spoken_languages, production_countries, production_companies same as genres

In [370]:
df.spoken_languages

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45461               [{'iso_639_1': 'fa', 'name': 'فارسی'}]
45462                    [{'iso_639_1': 'tl', 'name': ''}]
45463             [{'iso_639_1': 'en', 'name': 'English'}]
45464                                                   []
45465             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45466, dtype: object

In [371]:
df.spoken_languages = df.spoken_languages.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [372]:
df.spoken_languages

0                 English
1        English|Français
2                 English
3                 English
4                 English
               ...       
45461               فارسی
45462                    
45463             English
45464                    
45465             English
Name: spoken_languages, Length: 45466, dtype: object

In [373]:
df.spoken_languages.value_counts(dropna = False).head(10)

English             22395
                     3952
Français             1853
日本語                  1289
Italiano             1218
Español               902
Pусский               807
Deutsch               762
English|Français      681
English|Español       572
Name: spoken_languages, dtype: int64

In [374]:
df.spoken_languages.replace('', np.nan, inplace = True)

In [375]:
df.production_companies = df.production_companies.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)
df.production_countries = df.production_countries.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [376]:
df.production_companies.replace('', np.nan, inplace = True)
df.production_countries.replace('', np.nan, inplace = True)

###Number of missing values - isna().sum()

In [377]:
df.isna().sum()

belongs_to_collection    40975
budget                       0
genres                    2442
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3958
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [378]:
pd.read_csv('/content/drive/MyDrive/230222_Master 12 Advanced Projects/Project_03_Materials/movies_metadata.csv', low_memory = False).isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

##Handling the numerical columns

In [379]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

In [380]:
# df.budget.astype('float')

In [381]:
# pd.to_numeric(df.budget)

###The invalid parsing will be set as NaN - pd.to_numeric()

In [382]:
df.budget = pd.to_numeric(df.budget, errors = 'coerce')

In [383]:
df.budget.value_counts(dropna = False)

0.0           36573
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
9750000.0         1
7275000.0         1
78146652.0        1
280.0             1
1254040.0         1
Name: budget, Length: 1224, dtype: int64

###Replacing zeros with np.nan

In [384]:
df.budget = df.budget.replace(0, np.nan)

###Displaying values in millions

In [385]:
df.budget = df.budget.div(1000000)

In [386]:
df.revenue = df.revenue.replace(0, np.nan)
df.revenue = df.revenue.div(1000000)

In [387]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 8890 non-null   float64
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                7408 non-null   float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

###Renaming columns

In [388]:
df.rename(columns = {'revenue': 'revenue_musd', 'budget': 'budget_musd'}, inplace = True)

In [389]:
df.columns

Index(['belongs_to_collection', 'budget_musd', 'genres', 'id',
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue_musd', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [390]:
df.runtime.value_counts(dropna = False).head(10)

90.0     2556
0.0      1558
100.0    1470
95.0     1412
93.0     1214
96.0     1104
92.0     1080
94.0     1062
91.0     1057
88.0     1032
Name: runtime, dtype: int64

In [391]:
df.runtime = df.runtime.replace(0, np.nan)

In [392]:
# pd.to_numeric(df.id)

In [393]:
df.id = pd.to_numeric(df.id, errors = 'coerce')

In [394]:
df.id.value_counts(dropna = False).head(10)

NaN         3
141971.0    3
11115.0     2
25541.0     2
15028.0     2
132641.0    2
84198.0     2
13209.0     2
77221.0     2
152795.0    2
Name: id, dtype: int64

In [395]:
df.popularity = pd.to_numeric(df.popularity, errors = 'coerce')

In [396]:
df.popularity.value_counts(dropna = False).head(10)

0.000000    66
0.000001    56
0.000308    43
0.000220    40
0.000844    38
0.000578    38
0.001177    38
0.002001    28
0.003013    21
0.001393    19
Name: popularity, dtype: int64

In [397]:
df.vote_count.value_counts(dropna = False).head(10)

1.0    3264
2.0    3132
0.0    2899
3.0    2787
4.0    2480
5.0    2097
6.0    1747
7.0    1570
8.0    1359
9.0    1194
Name: vote_count, dtype: int64

In [398]:
df.vote_average.value_counts(dropna = False).head(10)

0.0    2998
6.0    2468
5.0    2001
7.0    1886
6.5    1722
6.3    1603
5.5    1381
5.8    1369
6.4    1350
6.7    1342
Name: vote_average, dtype: int64

###Deleting the value of 0 in vote_average column in the vote_count is equal to 0

In [399]:
df.loc[df.vote_count == 0, 'vote_average'] = np.nan

##Cleaning the datatime columns

In [400]:
df.release_date

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [401]:
# pd.to_datetime(df.release_date)

In [402]:
df.release_date = pd.to_datetime(df.release_date, errors = 'coerce')

In [403]:
df.release_date.value_counts(dropna = False).head(10)

2008-01-01    136
2009-01-01    121
2007-01-01    118
2005-01-01    111
2006-01-01    101
2002-01-01     96
2004-01-01     90
NaT            90
2001-01-01     84
2003-01-01     76
Name: release_date, dtype: int64

##Cleaning the text columns

In [404]:
df.original_language.value_counts(dropna = False).head(10)

en    32269
fr     2438
it     1529
ja     1350
de     1080
es      994
ru      826
hi      508
ko      444
zh      409
Name: original_language, dtype: int64

In [405]:
df.overview.value_counts(dropna = False).head(10)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               954
No overview found.                                                                                                                                                                                                                                                                                                                                                                                                                

In [406]:
no_overview = ['No overview found.', 'No Overview', 'No movie overview available.', 'No overview yet', ' ']

for text in no_overview:
  df.overview.replace(text, np.nan, inplace = True)

In [407]:
df.overview.value_counts(dropna = False).head(10)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1102
Recovering from a nail gun shot to the head and 13 months of coma, doctor Pekka Valinta starts to unravel the mystery of his past, still suffering from total amnesia.                                                                                                                                                                                                       

In [408]:
df.tagline.value_counts(dropna = False).head(10)

NaN                                   25054
Based on a true story.                    7
Trust no one.                             4
Be careful what you wish for.             4
-                                         4
Classic Albums                            3
Some doors should never be opened.        3
A Love Story                              3
Drama                                     3
Know Your Enemy                           3
Name: tagline, dtype: int64

In [409]:
df.tagline.replace('-', np.nan, inplace = True)

##Removing duplicates

###Display all duplicates - df.duplicated(keep = False)

In [410]:
df[df.duplicated(keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
7345,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
9165,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
24844,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
14012,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
22151,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,NaN,United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
14000,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,NaN,United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
8068,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,Filmfour,France|Germany|India|United Kingdom,2001-09-23,NaN,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
9327,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,Filmfour,France|Germany|India|United Kingdom,2001-09-23,NaN,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
17229,NaN,NaN,Drama,25541.0,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,NaN,Sweden|Denmark,2009-10-21,NaN,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0
23044,NaN,NaN,Drama,25541.0,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,NaN,Sweden|Denmark,2009-10-21,NaN,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0


In [411]:
df.drop_duplicates(inplace = True)

###Checking duplicated IDs

In [412]:
df[df.duplicated(subset = 'id', keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
33826,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",7.645827,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
5865,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",11.331072,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
4114,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,10.264597,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.0
44821,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,6.480376,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0
44826,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",6.080108,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,NaN,Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
5535,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",7.072301,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,NaN,Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
15765,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.529879,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,Jafar Panahi Film Productions,Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
11342,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.528960,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,Jafar Panahi Film Productions,Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
10419,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.185256,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,Magnolia Pictures|Extension 765,United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0
12066,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.008299,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,Magnolia Pictures|Extension 765,United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0


In [413]:
df.drop_duplicates(subset = 'id', keep = 'first', inplace = True)

In [414]:
df.id.value_counts(dropna = False).head(10)

862.0       1
74458.0     1
296206.0    1
107308.0    1
16247.0     1
99904.0     1
109463.0    1
116439.0    1
86541.0     1
53210.0     1
Name: id, dtype: int64

##Handling missing values - do nothing or remove rows

In [415]:
df.isna().sum()

belongs_to_collection    40946
budget_musd              36554
genres                    2442
id                           1
original_language           11
overview                  1102
popularity                   4
poster_path                386
production_companies     11872
production_countries      6283
release_date                88
revenue_musd             38036
runtime                   1819
spoken_languages          3954
status                      85
tagline                  25037
title                        4
vote_average              2900
vote_count                   4
dtype: int64

###Display rows without the title

In [416]:
df[df.title.isna()]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19729,NaN,NaN,Action|Thriller|Drama,82663.0,en,British soldiers force a recently captured IRA...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19730,NaN,NaN,Carousel Productions|Vision View Entertainment...,NaN,104.0,Released,NaN,Midnight Man,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29502,Mardock Scramble Collection,NaN,Animation|Science Fiction,122662.0,ja,Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,NaN,NaN,TV Movie|Action|Horror|Science Fiction,249260.0,en,A group of skiers are terrorized during spring...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [417]:
df.dropna(subset = ['title', 'id'], inplace = True)

###Converting id column to int

In [418]:
df.id = df.id.astype('int')

###Checking how many not missing values are in rows

In [419]:
df.notna().sum(axis = 1)

0        18
1        18
2        17
3        18
4        18
         ..
45461    14
45462    14
45463    16
45464    12
45465    12
Length: 45430, dtype: int64

In [420]:
df.notna().sum(axis = 1).value_counts().sort_values(ascending = False)

15    12522
16    11454
14     5424
17     4265
18     3859
13     3041
12     1890
19     1132
11     1020
10      512
9       183
8       104
7        20
6         4
dtype: int64

###Removing rows with less than 10 non-missing values

In [421]:
df[df.notna().sum(axis = 1) == 6]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
18038,NaN,NaN,NaN,344741,pl,NaN,0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Pajęczarki,NaN,0.0
20166,NaN,NaN,NaN,139909,fi,NaN,0.000127,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Aurinkotuuli,NaN,0.0
41718,NaN,NaN,NaN,216550,de,NaN,0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Dolpo Tulku - Heimkehr in den Himalaya,NaN,0.0
44978,NaN,NaN,NaN,398295,pt,NaN,0.000331,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Mundo Cão,NaN,0.0


In [422]:
df.dropna(thresh = 10, inplace = True)

In [423]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45119 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4487 non-null   object        
 1   budget_musd            8878 non-null   float64       
 2   genres                 42970 non-null  object        
 3   id                     45119 non-null  int64         
 4   original_language      45108 non-null  object        
 5   overview               44144 non-null  object        
 6   popularity             45119 non-null  float64       
 7   poster_path            44887 non-null  object        
 8   production_companies   33561 non-null  object        
 9   production_countries   39147 non-null  object        
 10  release_date           45079 non-null  datetime64[ns]
 11  revenue_musd           7398 non-null   float64       
 12  runtime                43552 non-null  float64       
 13  s

In [424]:
df.isna().sum()

belongs_to_collection    40632
budget_musd              36241
genres                    2149
id                           0
original_language           11
overview                   975
popularity                   0
poster_path                232
production_companies     11558
production_countries      5972
release_date                40
revenue_musd             37721
runtime                   1567
spoken_languages          3657
status                      66
tagline                  24723
title                        0
vote_average              2659
vote_count                   0
dtype: int64

##Final cleaning steps

In [425]:
df.status.value_counts()

Released           44692
Rumored              226
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [426]:
df.loc[df.status == 'Rumored', 'title']

682                  Butterfly Kiss
775            A Very Natural Thing
1032                  The Sunchaser
1478                The War at Home
1513                  Love Serenade
                    ...            
44655    The Surrender of Tournavos
44731             A Corner in Wheat
45030     The Story of Menstruation
45109     But What If This Is Love?
45289                   King's Road
Name: title, Length: 226, dtype: object

In [427]:
df = df.loc[df.status == 'Released'].copy()

In [429]:
df.drop(columns = ['status'], inplace = True)

In [432]:
cols = ['id', 'title', 'tagline', 'release_date', 'genres', 'belongs_to_collection', 'original_language', 'budget_musd', 'revenue_musd', 'production_companies', 
        'production_countries', 'vote_count','vote_average', 'popularity', 'runtime', 'overview', 'spoken_languages', 'poster_path']

In [433]:
df = df.loc[:, cols]

In [434]:
df

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,Pixar Animation Studios,United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,Warner Bros.|Lancaster Gate,United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,Twentieth Century Fox Film Corporation,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,Sandollar Productions|Touchstone Pictures,United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,/e64sOI48hQXyru7naBFyssKFxVd.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,Rising and falling between a man and woman,NaT,Drama|Family,NaN,fa,NaN,NaN,NaN,Iran,1.0,4.0,0.072051,90.0,Rising and falling between a man and woman.,فارسی,/jldsYflnId4tTWPx8es3uzsB1I8.jpg
45462,111109,Century of Birthing,NaN,2011-11-17,Drama,NaN,tl,NaN,NaN,Sine Olivia,Philippines,3.0,9.0,0.178241,360.0,An artist struggles to finish his work while a...,NaN,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg
45463,67758,Betrayal,A deadly game of wits.,2003-08-01,Action|Drama|Thriller,NaN,en,NaN,NaN,American World Pictures,United States of America,6.0,3.8,0.903007,90.0,"When one of her hits goes wrong, a professiona...",English,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg
45464,227506,Satan Triumphant,NaN,1917-10-21,NaN,NaN,en,NaN,NaN,Yermoliev,Russia,0.0,NaN,0.003503,87.0,"In a small town live two brothers, one a minis...",NaN,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg


In [435]:
df.reset_index(drop = True, inplace = True)

In [436]:
df

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,Pixar Animation Studios,United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,Warner Bros.|Lancaster Gate,United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,Twentieth Century Fox Film Corporation,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,Sandollar Productions|Touchstone Pictures,United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,/e64sOI48hQXyru7naBFyssKFxVd.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44687,439050,Subdue,Rising and falling between a man and woman,NaT,Drama|Family,NaN,fa,NaN,NaN,NaN,Iran,1.0,4.0,0.072051,90.0,Rising and falling between a man and woman.,فارسی,/jldsYflnId4tTWPx8es3uzsB1I8.jpg
44688,111109,Century of Birthing,NaN,2011-11-17,Drama,NaN,tl,NaN,NaN,Sine Olivia,Philippines,3.0,9.0,0.178241,360.0,An artist struggles to finish his work while a...,NaN,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg
44689,67758,Betrayal,A deadly game of wits.,2003-08-01,Action|Drama|Thriller,NaN,en,NaN,NaN,American World Pictures,United States of America,6.0,3.8,0.903007,90.0,"When one of her hits goes wrong, a professiona...",English,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg
44690,227506,Satan Triumphant,NaN,1917-10-21,NaN,NaN,en,NaN,NaN,Yermoliev,Russia,0.0,NaN,0.003503,87.0,"In a small town live two brothers, one a minis...",NaN,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg


In [440]:
df.poster_path[0]

"<img src='http://image.tmdb.org/t/p/w185//rhIRbceoE9lR4veEXuwCC2wARtG.jpg' style='height:100px;'>"

In [438]:
base_url = 'http://image.tmdb.org/t/p/w185/'
df.poster_path = "<img src='" + base_url + df.poster_path + "' style='height:100px;'>"

##Saving the df

In [439]:
df.to_csv('/content/drive/MyDrive/230222_Master 12 Advanced Projects/movies_cleaned.csv', index = False)